### 🧱 Module-Level Network Analysis + Robustness Summary

This script performs detailed **per-module** network topology evaluation and computes **global robustness metrics** for each group.

#### 📥 Input

- Network edge lists (`*_edges.csv`), one per group

#### 📤 Outputs

- `[group]_module_analysis.csv`: topology per Louvain module
- `network_robustness.csv`: group-wise summary of global metrics

#### 🧩 Per-module metrics:

| Metric                    | Description                                      |
|--------------------------|--------------------------------------------------|
| Nodes / Edges            | Size of the module                               |
| Density                  | Intra-module connection density                  |
| Avg. Degree              | Average weighted degree                          |
| Diameter                 | Longest shortest path in connected subgraph      |
| Clustering Coefficient   | Local clustering tendency                        |
| Avg. Path Length         | Average shortest path (if connected)             |
| Betweenness Centrality   | Node bridging within module                      |
| Eigenvector Centrality   | Influence of nodes relative to others            |

#### 🛡️ Network Robustness Metrics:

| Metric              | Description                              |
|---------------------|------------------------------------------|
| Network Efficiency  | Global information propagation ability   |
| Average Degree      | Overall connectivity                     |

> This module supports group comparisons, biomarker prioritization, and robustness interpretation.


In [ ]:
import os
import networkx as nx
import community.community_louvain as community_louvain
import pandas as pd

# Define input/output paths
input_dir = r'..group results'  # Directory containing *_edges.csv
output_dir = r'..group_network_analysis'  # Per-module outputs
robustness_output_file = os.path.join(output_dir, 'network_robustness.csv')  # Global robustness summary
os.makedirs(output_dir, exist_ok=True)

# Get all network edge files
network_files = [f for f in os.listdir(input_dir) if f.endswith('_edges.csv')]

# Container for robustness summary
robustness_data = []

# Loop through all group networks
for edges_file in network_files:
    group_name = edges_file.split('_edges.csv')[0]
    edges_file_path = os.path.join(input_dir, edges_file)
    output_file_path = os.path.join(output_dir, f'{group_name}_module_analysis.csv')

    # 1. Load edges and build graph
    edges_df = pd.read_csv(edges_file_path)
    G = nx.Graph()
    for _, row in edges_df.iterrows():
        G.add_edge(row['Node1'], row['Node2'], weight=row['Weight'])

    # 2. Make all weights positive
    for u, v, data in G.edges(data=True):
        if data['weight'] < 0:
            data['weight'] = abs(data['weight'])

    # 3. Louvain community detection
    partition = community_louvain.best_partition(G, weight='weight')

    # Group nodes by module
    modules = {}
    for node, module in partition.items():
        modules.setdefault(module, []).append(node)

    # 4. Calculate per-module topology
    module_data = []
    for module, nodes in modules.items():
        subgraph = G.subgraph(nodes)

        num_nodes = subgraph.number_of_nodes()
        num_edges = subgraph.number_of_edges()
        density = nx.density(subgraph)
        avg_degree = sum(dict(subgraph.degree(weight='weight')).values()) / num_nodes
        diameter = nx.diameter(subgraph) if nx.is_connected(subgraph) else None
        avg_clustering = nx.average_clustering(subgraph)
        avg_path_length = nx.average_shortest_path_length(subgraph) if nx.is_connected(subgraph) else None
        avg_betweenness = sum(nx.betweenness_centrality(subgraph, weight='weight').values()) / num_nodes
        avg_eigenvector = sum(nx.eigenvector_centrality(subgraph, weight='weight').values()) / num_nodes

        module_data.append({
            "Module": module,
            "Nodes": num_nodes,
            "Edges": num_edges,
            "Density": density,
            "Average Degree": avg_degree,
            "Diameter": diameter,
            "Clustering Coefficient": avg_clustering,
            "Average Path Length": avg_path_length,
            "Betweenness Centrality": avg_betweenness,
            "Eigenvector Centrality": avg_eigenvector
        })

    # Save module-level metrics
    module_df = pd.DataFrame(module_data)
    module_df.to_csv(output_file_path, index=False)
    print(f"{group_name} module-level analysis saved to: {output_file_path}")

    # 5. Compute global robustness metrics
    efficiency = nx.global_efficiency(G)
    avg_degree = sum(dict(G.degree(weight='weight')).values()) / len(G.nodes())
    robustness_data.append({
        "Group": group_name,
        "Network Efficiency": efficiency,
        "Average Degree": avg_degree
    })

# Save robustness summary
robustness_df = pd.DataFrame(robustness_data)
robustness_df.to_csv(robustness_output_file, index=False)
print(f"Network robustness summary saved to: {robustness_output_file}")

print("All group-level module and robustness analyses complete.")


### 🌐 Full Network and Module Analysis Pipeline

This script performs **Louvain community detection**, **node centrality calculation**, **per-module topological characterization**, and **network visualizations** for each group.

#### 📤 Outputs per group:
- `[group]_network_analysis.csv` – Node-level centralities and module labels
- `[group]_module_analysis.csv` – Module-level topological properties
- `group_network_visualizations/[group]_network.png` – PNG network diagram
- `network_robustness.csv` – Global network metrics for comparison

#### 📌 Key features:
- Corrects negative edge weights
- Calculates centralities: Degree, Betweenness, Eigenvector
- Computes module metrics: density, clustering, path length, etc.
- Saves full results and visualizations to disk

> Easily extendable to compare between timepoints, experimental conditions, or perform longitudinal tracking.


In [ ]:
import os
import networkx as nx
import pandas as pd
import numpy as np
import community.community_louvain as community_louvain
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.lines import Line2D

# Input and output directories
input_dir = r'..group results'
output_dir = input_dir  # Reuse same path for results
output_images_dir = os.path.join(output_dir, 'group_network_visualizations')
robustness_output_file = r'..network_robustness.csv'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)

# Get all edge files
network_files = [f for f in os.listdir(input_dir) if f.endswith('_edges.csv')]

# Store network robustness results
robustness_data = []

# Define drawing function
def draw_group_network(G, group_name, partition, pos, output_path):
    unique_modules = set(partition.values())
    color_palette = cm.get_cmap('plasma')
    module_colors = {
        module: color_palette(i / (len(unique_modules) - 1) * 0.7 + 0.3)
        for i, module in enumerate(sorted(unique_modules))
    }
    node_colors = [module_colors[partition[node]] for node in G.nodes()]
    node_sizes = [G.degree(node, weight='weight') * 3 for node in G.nodes()]
    edge_colors = [
        module_colors[partition[u]] if partition[u] == partition[v] else "black"
        for u, v in G.edges()
    ]
    edge_widths = [0.02 for _ in G.edges()]

    # Plot network
    plt.figure(figsize=(15, 15))
    ax = plt.gca()
    nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, alpha=0.9, ax=ax)
    nx.draw_networkx_edges(G, pos, width=edge_widths, edge_color=edge_colors, alpha=0.7, ax=ax)

    legend_elements = [
        Line2D([0], [0], marker='o', color='w', label=f'Module {module}',
               markerfacecolor=module_colors[module], markersize=10)
        for module in unique_modules
    ]
    plt.legend(handles=legend_elements, loc='upper right', title='Modules')
    plt.title(f"Network Visualization: {group_name}", fontsize=20)
    plt.axis('off')
    plt.savefig(output_path, format='png', dpi=300, bbox_inches='tight')
    plt.close()

# Loop through groups
for edges_file in network_files:
    group_name = edges_file.split('_edges.csv')[0]
    edges_file_path = os.path.join(input_dir, edges_file)
    output_file_path = os.path.join(output_dir, f'{group_name}_network_analysis.csv')
    output_image_path = os.path.join(output_images_dir, f"{group_name}_network.png")
    output_module_file_path = os.path.join(output_dir, f'{group_name}_module_analysis.csv')

    # Load edge data and build graph
    edges_df = pd.read_csv(edges_file_path)
    G = nx.Graph()
    for _, row in edges_df.iterrows():
        G.add_edge(row['Node1'], row['Node2'], weight=row['Weight'])
    for u, v, data in G.edges(data=True):
        if data['weight'] < 0:
            data['weight'] = abs(data['weight'])

    # Community detection
    partition = community_louvain.best_partition(G, weight='weight')
    nx.set_node_attributes(G, partition, 'Module')

    # Centrality metrics
    degree_centrality = nx.degree_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G, weight='weight')
    eigenvector_centrality = nx.eigenvector_centrality(G, weight='weight', max_iter=1000)

    # Global robustness metrics
    efficiency = nx.global_efficiency(G)
    avg_degree = sum(dict(G.degree(weight='weight')).values()) / len(G.nodes())

    robustness_data.append({
        "Group": group_name,
        "Network Efficiency": efficiency,
        "Average Degree": avg_degree
    })

    # Save node-level analysis
    data = []
    for node in G.nodes():
        data.append({
            "Node": node,
            "Module": partition[node],
            "Degree Centrality": degree_centrality[node],
            "Betweenness Centrality": betweenness_centrality[node],
            "Eigenvector Centrality": eigenvector_centrality[node]
        })
    df = pd.DataFrame(data)
    df.to_csv(output_file_path, index=False)

    # Draw network layout
    pos = nx.fruchterman_reingold_layout(G, seed=42, k=5, iterations=500, scale=7)
    draw_group_network(G, group_name, partition, pos, output_image_path)
    print(f"{group_name} network analysis and image saved to: {output_file_path}, {output_image_path}")

    # Module-level analysis
    modules = {}
    for node, module in partition.items():
        modules.setdefault(module, []).append(node)

    module_data = []
    for module, nodes in modules.items():
        subgraph = G.subgraph(nodes)
        num_nodes = subgraph.number_of_nodes()
        num_edges = subgraph.number_of_edges()
        density = nx.density(subgraph)
        avg_degree = sum(dict(subgraph.degree(weight='weight')).values()) / num_nodes
        diameter = nx.diameter(subgraph) if nx.is_connected(subgraph) else None
        avg_clustering = nx.average_clustering(subgraph)
        avg_path_length = nx.average_shortest_path_length(subgraph) if nx.is_connected(subgraph) else None
        avg_betweenness = sum(nx.betweenness_centrality(subgraph, weight='weight').values()) / num_nodes
        avg_eigenvector = sum(nx.eigenvector_centrality(subgraph, weight='weight').values()) / num_nodes

        module_data.append({
            "Module": module,
            "Nodes": num_nodes,
            "Edges": num_edges,
            "Density": density,
            "Average Degree": avg_degree,
            "Diameter": diameter,
            "Clustering Coefficient": avg_clustering,
            "Average Path Length": avg_path_length,
            "Betweenness Centrality": avg_betweenness,
            "Eigenvector Centrality": avg_eigenvector
        })

    module_df = pd.DataFrame(module_data)
    module_df.to_csv(output_module_file_path, index=False)
    print(f"{group_name} module-level analysis saved to: {output_module_file_path}")

# Save robustness summary
robustness_df = pd.DataFrame(robustness_data)
robustness_df.to_csv(robustness_output_file, index=False)
print(f"Network robustness summary saved to: {robustness_output_file}")
print("All network analyses and visualizations complete.")
